## Preparation and Connecting to ICW:

In [1]:
# package required to connect to ICW
import cadspy

# Import other packages
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# diplay all rows and cols when using 'dataframe'.head() or 'dataframe'.tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
username = str(input('User number'))
icw = cadspy.DatabaseConnection(system='ICW', user=username)

User number u243700
Enter Password:  ··············


<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:salmon;font-family:Verdana,sans-serif;font-size:16px;">

<font size="4">Can't connect to ICW? Instructions on how to get access to it in the links below:

</font>
</div>

- Step 1: [Requesting Access to GitHub](https://baplc.sharepoint.com/sites/ask/SitePages/Requesting-Access-to-GitHub.aspx)

Once your access to GitHub has been aproved, you need to:

- Step 2: [Request access to British-Ent GitHub organisation](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/Corporate_Directory_git.md)

And

- Step 3: [Setup Sagemaker Studio to access GitHub](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/sagemaker_to_github_setup.md)

<br>

<br>

### Data

#### S19 Lounge Eligibility data

In [3]:
query = """

sel * from LDB_SBOX_OR.HACKATHON_OPS_LOUNGE_ELIGIBILITY

"""

df_lounge_eligibility = icw.queryToDataframe(query)

In [4]:
print(df_lounge_eligibility.shape)
df_lounge_eligibility.head(2)

(1534750, 12)


,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13


#### S19 Flight info

In [5]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_FLIGHT_INFO

"""

df_flight_info = icw.queryToDataframe(query)

In [6]:
print(df_flight_info.shape)
df_flight_info.head(2)

(136219, 11)


,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE
0,2023-04-30 20:25:00,2023-04-30 20:34:00,BA,57,LHR,5,JNB,JNB,388,A8,LHRJNB
1,2019-05-18 12:15:00,2019-05-18 12:16:00,BA,632,LHR,5,ATH,ATH,320,R3,LHRATH


#### Station Code Decode

In [7]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_COUNTRY_DECODE

"""

df_country = icw.queryToDataframe(query)

In [8]:
print(df_country.shape)
df_country.head(2)

(159, 5)


,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,LHRINV,GB,United Kingdom and Northern Ireland,UK,UK
1,LHRSVO,RU,Russia in Europe,EASTERN EUROPE,EUROPE EXC UK


#### Additional Station Decodes from ICW reference table

There are lots of destinations which are in lounge_elig but not in df_country.
They are all new (since ~2019) routes at Heathrow.

We can access this information from an ICW table. We load that ICW reference table and filter for results not in our merged table, but that are in lounge_elig.

This should result in zero nan values arising from missing country information

In [9]:
query = """
SELECT STN_CD, COUNTRY_CD, COUNTRY_NM, CORP_GEOG_CTRY_GRP_NM, CORP_GEOG_CONTINENT_NM
FROM REF_GEOG_LOC_HIERARCHY
"""
df_additional_country_decodes=icw.queryToDataframe(query)

#### Aircraft Type

In [10]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_AC_TYPE

"""

df_acft_typ = icw.queryToDataframe(query)

In [11]:
df_acft_typ

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235
2,339,W9,WB,0,30,21,234
3,777,K7,WB,0,48,24,203
4,781,X8,WB,7,49,35,165
5,351,K5,WB,0,56,56,219
6,321,M6,NB,0,23,0,131
7,32Q,N6,NB,0,32,0,172
8,789,L8,WB,8,42,39,127
9,32A,H3,NB,0,20,0,150


<br>

### Pre-processing

*Hint:* It is always worth checking the format of each of the columns in your dataframes before trying to do any work with them. To do so, you can make use of the `headers_and_first_row` function below.


In [12]:
def headers_and_first_row(df):
    '''
    print headers and first row of a df to deal with data types
    '''
    
    headers = df.columns
    first_row = []

    for col in headers:
        first_row.append(df[col][0])
    
    dictionary = dict( zip( headers, first_row) )

    return dictionary

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 1:** Pre-process **all** the tables above (df_lounge_eligibility, df_flight_info, df_country, df_acft_typ, df_additional_country_decodes).

</font>

</ol>
</div>

We will first strip all frames of spaces. We will then go through each frame to properly prepare it.

We will then load some additional datasets that will be useful for analysis.

In [13]:
# Stripping strings.
# Define the function to strip strings from the tables.

# Define a function that fine all string fields and remove all blak spaces
def data_cleaning_string(df):
    # Get names of all fields in a dataframe
    fields = df.columns
    # loop for all fields, if data type is string then remove blank spaces
    for f in fields:
        if type(df[f][0]) == str:
            df[f] = df[f].str.strip()
    return df

In [14]:
tables = [df_lounge_eligibility,df_flight_info,df_country,df_acft_typ,df_additional_country_decodes]
for i,t in enumerate(tables):
    tables[i] = data_cleaning_string(t)

Clean the flight info dataset.

In [15]:
# We remove duplicates in flight info.
# Firstly, we sort by planned departure date, flight number and actual departure station.
df_flight_info = df_flight_info.sort_values(['GMT_PLND_DEP_TS','OPG_FLT_NO','GMT_ACT_DEP_TS'])
# Remove all duplicates and keep the first rows of all duplicated.
# Duplicated flights arise when e.g. flights return to LHR. We want to keep the flight as originally planned as this is represented in the lounge eligibility dataset.
df_flight_info = df_flight_info.drop_duplicates(subset = ['OPG_FLT_NO','GMT_PLND_DEP_TS'],keep = "first")

In [16]:
# Create separate columns for date and time within df_flight_info.

df_flight_info['GMT_PLND_DEP_DT'] = df_flight_info['GMT_PLND_DEP_TS'].dt.date
df_flight_info['GMT_PLND_DEP_TIME'] = df_flight_info['GMT_PLND_DEP_TS'].dt.time

Clean the country information dataset.

In [17]:
# Create separate columns for the departure and arrival airports. This is needed when joining to lounge eligibility.
df_country['DEP_STN_CD'] = df_country['ROUTE'].str.slice(0,3)
df_country['ARR_STN_CD'] = df_country['ROUTE'].str.slice(3,6)

Clean the aircraft type dataset

In [18]:
# Change seat quantities to integers.
df_acft_typ['FIRST_SEATS_QTY'] = df_acft_typ['FIRST_SEATS_QTY'].astype('int')
df_acft_typ['CLUB_SEATS_QTY'] = df_acft_typ['CLUB_SEATS_QTY'].astype('int')
df_acft_typ['PREM_ECONOMY_SEATS_QTY'] = df_acft_typ['PREM_ECONOMY_SEATS_QTY'].astype('int')
df_acft_typ['ECONOMY_SEATS_QTY'] = df_acft_typ['ECONOMY_SEATS_QTY'].astype('int')

df_acft_typ.head(1)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132


<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 2:** Join the tables below
   
    - df_flight_info
    - df_country
    - df_acft_typ
    - df_additional_country_decodes
    
to the table df_lounge_eligibility to generate a final dataset.

</font>

</ol>
</div>

In [19]:
# We first merge flight information onto the lounge eligibility dataset.
# We merge on three common characteristics: Operating Airline Code, flight number and departure date.
# We use left merge: this retains all lounge_elig rows are retained, albeit with NaNs.

df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['OPERATING_AIRLINE_CD','OPERATING_FLT_NO','GMT_UPLIFT_DT'],
                                     right_on = ['OPG_ALN_CD','OPG_FLT_NO','GMT_PLND_DEP_DT'] ,
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

# display the merged table.
df_lounge_elig_flight_info.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00


Get rows with any null values

In [20]:
# There are currently 1183 rows for which flight information is not available.
print(df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)].shape)

# This represents 357 missing flights.
print(len(df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)]['Skew_Id'].unique()))

(1183, 25)
357


### Merge `df_country` to table

Begin with cleaning the additional country decode information.

In [22]:
# For efficiency, we should merge df_country and df_additional_country_decodes first.
# Firstly, organise df_additional_country_decodes to have the same columns.
# if statement to avoid throwing errors if column renaming from cell below has already been done.

if 'STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'STN_CD'
elif 'ARR_STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'ARR_STN_CD'
df_additional_country_decodes = df_additional_country_decodes[~df_additional_country_decodes[station_code_col].isin(df_country['ARR_STN_CD'])]

# for more efficiency, we can remove all those rows which are not needed as there are no lounge elig rows with that destinations.

df_additional_country_decodes = df_additional_country_decodes[df_additional_country_decodes[station_code_col].isin(df_lounge_elig_flight_info['DISCHARGE_STN_CD'])]
print(df_additional_country_decodes.shape)
# This leaves 30 destinations.

(30, 5)


In [23]:
# We need df_additional_country_decodes to have the same column names and order as df_country, ready for pd.concat.
# By merging now, we eliminate the need to merge separately with df_lounge_elig.

df_additional_country_decodes['ROUTE'] = 'LHR' + df_additional_country_decodes[station_code_col]
df_additional_country_decodes['DEP_STN_CD'] = 'LHR'

# We must ensure column names and order of df_additional_country_decodes match df_country.

if 'STN_CD' in list(df_additional_country_decodes.columns):

    df_additional_country_decodes.rename(columns={'STN_CD':'ARR_STN_CD'},inplace=True)
df_additional_country_decodes = df_additional_country_decodes[df_country.columns]

In [24]:
# concat the two country tables together.
df_country = pd.concat([df_country,df_additional_country_decodes])

In [25]:
# merge country info to the combined dataset. 
# We merge on a column originally derived from lounge_elig. This ensures there are no missing values due to missing flight information.

df_lounge_elig_flight_info_country = pd.merge(df_lounge_elig_flight_info,# left table
                                     df_country, # right table
                                     left_on = ['DISCHARGE_STN_CD'],
                                     right_on = ['ARR_STN_CD'] ,
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

df_lounge_elig_flight_info_country.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI


In [26]:
df_lounge_elig_flight_info_country.shape

(1534750, 32)

Check for null values

In [27]:
df_lounge_elig_flight_info_country[df_lounge_elig_flight_info_country['CORP_GEOG_CONTINENT_NM'].isna()].head(5)
# There are no null values arising from lack of country data.

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD


### `df_acft_typ`

In [28]:
# merge aircraft type into the combined country/flight info/lounge_elig dataset.
# We have to merge on a flight_info column. This means all rows with missing flight information will not get aircraft type information.

df_lounge_elig_flight_info_country_acft_typ = pd.merge(
    df_lounge_elig_flight_info_country,
    df_acft_typ,
    left_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    right_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    how = 'left'
)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NB,0.0,21.0,0.0,113.0
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR,NB,0.0,21.0,0.0,113.0
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL,NB,0.0,20.0,0.0,150.0
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE,NB,0.0,16.0,0.0,119.0
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI,WB,0.0,45.0,0.0,212.0


In [29]:
df_lounge_elig_flight_info_country_acft_typ.shape

(1534750, 37)

In [30]:
df_acft_typ

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235
2,339,W9,WB,0,30,21,234
3,777,K7,WB,0,48,24,203
4,781,X8,WB,7,49,35,165
5,351,K5,WB,0,56,56,219
6,321,M6,NB,0,23,0,131
7,32Q,N6,NB,0,32,0,172
8,789,L8,WB,8,42,39,127
9,32A,H3,NB,0,20,0,150


Check for null

In [31]:
df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
300,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,SAPP,Tier 3,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
466,2019-09-298598,BA,8598,2019-09-29,LHR,CFU,M,M,,,Not eligible,43,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRCFU,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,CFU,NaN,NaN,NaN,NaN,NaN
876,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,,Tier 3,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
1452,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,,,Not eligible,144,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
11711,2019-06-168586,BA,8586,2019-06-16,LHR,KLX,M,M,Gold,EMER,Tier 2,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRKLX,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,KLX,NaN,NaN,NaN,NaN,NaN


In [32]:
df_lounge_elig_flight_info_country_acft_typ.loc[(df_lounge_elig_flight_info_country_acft_typ['IATA_AC_TYP_CD'].notna())&(df_lounge_elig_flight_info_country_acft_typ['WB_NB_CAT'].isna())]['ACT_AC_TYP_CD'].unique()

# We are missing two aircraft types: T7 and U7.

# If you want more info, run the line below.
# df_flight_info.loc[df_flight_info['ACT_AC_TYP_CD'].isin(['T7','U7'])]
# The two missing aircraft types are both 777. There are only 29 flight affected, so we can proceed without them when analysing by aircraft type.

array(['T7', 'U7'], dtype=object)

### More cleaning for columns redundancy

In [33]:
# drop redundant columns.

df_final = df_lounge_elig_flight_info_country_acft_typ.copy()

to_drop = ['GMT_PLND_DEP_TS','GMT_ACT_DEP_TS','OPG_ALN_CD','OPG_FLT_NO','ACT_DEP_STN_CD','PLND_ARR_STN_CD','ACT_ARR_STN_CD','ROUTE_x',
          'GMT_PLND_DEP_DT','ROUTE_y','DEP_STN_CD','ARR_STN_CD']

df_final = df_final.drop(columns = to_drop,)
df_final['GMT_UPLIFT_YR'] = [dts.year for dts in df_final['GMT_UPLIFT_DT']]

print(df_final.shape)
df_final.head(5)

(1534750, 26)


,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,2023
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,3,319,M4,16:35:00,DE,Germany,WEST EUROPE,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,2023
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,5,32A,H3,18:25:00,NO,Norway,SCANDINAVIA,EUROPE EXC UK,NB,0.0,20.0,0.0,150.0,2019
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,5,319,A4,14:40:00,FR,France,WEST EUROPE,EUROPE EXC UK,NB,0.0,16.0,0.0,119.0,2019
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,5,343,W9,16:15:00,EG,Egypt,NORTH AFRICA,AFRICA,WB,0.0,45.0,0.0,212.0,2019


<br><br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">Exercise 3: Based on S2019/S2023 data, provide a lookup table of Lounge eligibility assumptions that can be applied to a future schedule. To do so, answer each of the following questions in the Markdown cell provided below. 

- What level of granularity do you use?
- What metric do you use to come up with Lounge eligibility profiles?

    
Note 1: **Provide evidence for your assumptions.** This can be in the form of tables, graphs, correlation matrix, etc.
    
Note 2: Make use of the examples below to give structure to your answer. Feel free to attend the Hackathon Clinics if you have any questions. 
</font>


    
</ol>
</div>

**Reasoning (Example 1)**:


Assume:
- Data has been preprocessed.
- Data has been joined, and a final dataset has been created. This dataset is the result of joining the 4 tables.

The final dataset has been called `df_lounge_elig_flight_info_country_acft_typ`.

<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their aircraft type. I will therefore have a lookup table with two categories: Narrowbody and Widebody. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by Aircraft Type, by Tier. Then I have divided them by the total number of passengers by Aircraft Type. For example:
    - For NB aircrafts, and for Tier 1 passengers: In S19 we had 41,728 pax eligible for Tier 1 out of 7,222,830 pax flying on Narrowbody aircraft. This represents 0.6% of the costumers and I assume that this will be the number of costumers elegible for this specific Lounge in a future schedule.   

In [34]:
# your code here!

In [35]:
# columns that you want to group by
list_groupby = ['WB_NB_CAT','Lounge_eligibility_tier']

# grouping by WB_NB_CAT and Lounge_eligibility_tier
df_groupby_wb_nb = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

In [36]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count
0,NB,Not eligible,9741188
1,NB,Tier 1,53292
2,NB,Tier 2,744829
3,NB,Tier 3,2240579
4,WB,Not eligible,5730922
5,WB,Tier 1,236775
6,WB,Tier 2,391264
7,WB,Tier 3,1523611


In [37]:
# getting the number of pax by aircraft type

# columns that you want to group by
list_groupby = ['WB_NB_CAT']

# grouping by WB_NB_CAT
df_groupby_wb_nb_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [38]:
# a look at the data
df_groupby_wb_nb_ttl

,WB_NB_CAT,pax_count_ttl
0,NB,12779888
1,WB,7882572


In [39]:
# now, let's left join df_groupby_wb_nb_ttl onto df_groupby_wb_nb
# this adds a new column to the df_groupby_wb_nb table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_wb_nb = pd.merge(df_groupby_wb_nb,
                            df_groupby_wb_nb_ttl,
                            on = ['WB_NB_CAT'],
                            how = 'left'
                           )

In [40]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,NB,Not eligible,9741188,12779888
1,NB,Tier 1,53292,12779888
2,NB,Tier 2,744829,12779888
3,NB,Tier 3,2240579,12779888
4,WB,Not eligible,5730922,7882572
5,WB,Tier 1,236775,7882572
6,WB,Tier 2,391264,7882572
7,WB,Tier 3,1523611,7882572


In [41]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_wb_nb['pax_eligible%'] = (df_groupby_wb_nb['pax_count'] / df_groupby_wb_nb['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_wb_nb['pax_eligible%'] = df_groupby_wb_nb['pax_eligible%'].map('{:,.1f}%'.format)

In [42]:
# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_wb_nb.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [43]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_eligible%
0,NB,Not eligible,76.2%
1,NB,Tier 1,0.4%
2,NB,Tier 2,5.8%
3,NB,Tier 3,17.5%
4,WB,Not eligible,72.7%
5,WB,Tier 1,3.0%
6,WB,Tier 2,5.0%
7,WB,Tier 3,19.3%


In [44]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_wb_nb['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_wb_nb = df_groupby_wb_nb[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [45]:
# using set_index to come up with the final lookup table
df_groupby_wb_nb = df_groupby_wb_nb.set_index(['WB_NB_CAT','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [46]:
# final table
df_groupby_wb_nb

pax_eligible%              
Lounge_eligibility_tier        Tier 1 Tier 2 Tier 3
WB_NB_CAT                                          
NB                               0.4%   5.8%  17.5%
WB                               3.0%   5.0%  19.3%

<br>

**Feedback:** This analysis provides a lookup table in the format needed to be input into a future schedule. Nevertheless, this analysis is too high level and you haven't provided any evidence for your assumptions. To further enhance your answer use insights from the data and provide evidence for your assumptions. Please find some ideas below: 

- Using the same categories (WB,NB), plot data overtime to better understand the peaks for the different lounges. 
- Is there a way to split Widebody into more categories? Do the Haul, Region, Time of Day, or Country play a role in the number of passengers that are eligible in Tier 1, Tier 2 and Tier 3? Etc...

<br><br>

**Reasoning (Example 2)**:



<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their flight number. I will therefore have a lookup table with a lot of categories as each flight number is a category. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by flight number, by Tier. Then I have divided them by the total number of passengers flight number.  

In [47]:
# your code here!

In [48]:
# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_flt_no = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count
0,5,HND,Not eligible,15825
1,5,HND,Tier 1,220
2,5,HND,Tier 2,888
3,5,HND,Tier 3,4469


In [49]:
# getting the number of pax by OPERATING_FLT_NO and DISCHARGE_STN_CD

# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD']

# grouping by WB_NB_CAT
df_groupby_flt_no_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [50]:
# a look at the data
df_groupby_flt_no_ttl.head(2)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,pax_count_ttl
0,5,HND,21402
1,5,NRT,38378


In [51]:
# now, let's left join df_groupby_flt_no_ttl onto df_groupby_flt_no
# this adds a new column to the df_groupby_flt_no table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_flt_no = pd.merge(df_groupby_flt_no,
                            df_groupby_flt_no_ttl,
                            on = ['OPERATING_FLT_NO','DISCHARGE_STN_CD'],
                            how = 'left'
                           )

In [52]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,5,HND,Not eligible,15825,21402
1,5,HND,Tier 1,220,21402
2,5,HND,Tier 2,888,21402
3,5,HND,Tier 3,4469,21402


In [53]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_flt_no['pax_eligible%'] = (df_groupby_flt_no['pax_count'] / df_groupby_flt_no['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_flt_no['pax_eligible%'] = df_groupby_flt_no['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_flt_no.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [54]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_eligible%
0,5,HND,Not eligible,73.9%
1,5,HND,Tier 1,1.0%
2,5,HND,Tier 2,4.1%
3,5,HND,Tier 3,20.9%


In [55]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_flt_no['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_flt_no = df_groupby_flt_no[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of customers as columns.

In [56]:
# using set_index to come up with the final lookup table
df_groupby_flt_no = df_groupby_flt_no.set_index(['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [57]:
# final table
df_groupby_flt_no.head(4)

pax_eligible%              
Lounge_eligibility_tier                  Tier 1 Tier 2 Tier 3
OPERATING_FLT_NO DISCHARGE_STN_CD                            
5                HND                       1.0%   4.1%  20.9%
                 NRT                       3.6%   4.8%  19.1%
7                HND                       3.3%   5.2%  19.8%
9                BKK                       0.3%   4.2%  15.9%

<br>

**Feedback:** This analysis goes at a very granular level, we might come up with missing values if we apply this lounge eligibility profiles to a future schedule. Here are some ideas to further enhance your answer: 

- What would happen if we fly to a new destination in the future? How do we ensure we have a lounge eligibility profile for this new route?
- As you can see in the example above: Pax eligible for Tier 1 for the BKK flight is significantly different from the rest. What's the most used aircraft type for this route? And why it differs that much from the rest? Is it because of the route characteristics instead?
- For SH routes, we might change the time of departure for a specific flight number from one year to the next. Explore the possibility of using a clasification that takes that into account, like using Time of Day instead of flight number. 

## Additional data preparation.
Our analysis requires some additional data preparation, shown in this section.

Firstly, we would like to be able to split the data into 2019/23 data separately, which can be done by adding a year field into the table.

In [58]:
# First, convert uplift date to datetime object.

df_final['GMT_UPLIFT_DT'] = pd.to_datetime(df_final['GMT_UPLIFT_DT'])

# We would also want to get the uplift year

df_final['GMT_UPLIFT_YR'] = df_final['GMT_UPLIFT_DT'].dt.year

In [59]:
# create a function to work out the OneWorld status category of each passenger group.
# The current dataset is not perfect: many BA status passengers are not correctly reported as having OneWorld status.

def status_column_creator(df):
    
    if 'STATUS' not in list(df.columns):
        df['STATUS'] = 'NA'
    
    ba_emerald_list = ['Gold','Gold For Life','Gold Guest List','Gold Guest List For Life','Premier']

    # calculate how many are of each OneWorld Status level. Order of operations is important, highest reported status prevails.
    df.loc[(df['BA_PAX_TIER'].isin(['None','']))|(df['ONEWORLD_TIER']==''),'STATUS']='None'
    df.loc[(df['BA_PAX_TIER']=='Silver')|(df['ONEWORLD_TIER']=='SAPP'),'STATUS']='Sapphire'
    df.loc[(df['BA_PAX_TIER'].isin(ba_emerald_list))|(df['ONEWORLD_TIER']=='EMER'),'STATUS']='Emerald'

    # set BA column only. If blank, we want it to read none.
    df.loc[(df['BA_PAX_TIER']==''),'BA_PAX_TIER']='None'
    
    return df

In [60]:
# run this function with our dataset.
df_final = status_column_creator(df_final)

In [61]:
# Create a function to work out why a passenger has their tier of lounge access.
# E.g. If a passenger is BA Silver and travels in First, they have Tier 1 lounge access from their travel cabin.
# E.g. If a OneWorld Emerald is travelling in World Traveller Plus, they have Tier 2 lounge access from their status.
# E.g. If a OneWorld Sapphire is travelling in Club Europe, they have Tier 3 lounge access from both their status and their cabin.
# Note: the original lounge eligibility dataset only uses Travel Cabin to calculate lounge eligibility, not booked cabin. This affects e.g. downgrades.

def tier_source_finder(df):

    # create two columns indicating whether pax sources lounge eligibility from cabin and/or status. Default to No.
    df['SOURCE_CABIN_IND'] = 'N'
    df['SOURCE_STATUS_IND'] = 'N'
    
    # if pax in Tier 1, are they flying in F? If so set cabin source indicator to Yes (Y).
    tier_1_cabin_y = (df['Lounge_eligibility_tier']=='Tier 1') & ((df['TRAVEL_CABIN_CD']=='F'))
    df.loc[tier_1_cabin_y,['SOURCE_CABIN_IND']] = 'Y'
    
    # if they are in Tier 1, do they have the status for Tier 1 lounge access. If so, set cabin source indicator to Yes (Y).

    tier_1_status_list = ['Gold Guest List','Gold Guest List For Life','Premier']
    tier_1_status_y = (df['Lounge_eligibility_tier']=='Tier 1') & ((df['BA_PAX_TIER'].isin(tier_1_status_list)))
    df.loc[tier_1_status_y,['SOURCE_STATUS_IND']] = 'Y'
    
    # if they are in Tier 2, do they have the status for Tier 2 lounge access, either from BA or OneWorld.
    # We will not use the STATUS column. This is Jamie's custom function, best not to use in case that doesn't make it to final notebook. Instead, check both BA and OneWorld Tiers.
    
    tier_2_status_list = ['Gold','Gold For Life','EMER']
    tier_2_status_y = (df['Lounge_eligibility_tier']=='Tier 2') & ((df['BA_PAX_TIER'].isin(tier_2_status_list)|(df['ONEWORLD_TIER'].isin(tier_2_status_list))))
    df.loc[tier_2_status_y,['SOURCE_STATUS_IND']] = 'Y'
    
    # If they are in Tier 3, do they have the cabin for Tier 3 lounge access?
    tier_3_cabins_list = ['C','J']
    tier_3_cabins_y = (df['Lounge_eligibility_tier']=='Tier 3') & ((df['TRAVEL_CABIN_CD'].isin(tier_3_cabins_list)))
    df.loc[tier_3_cabins_y,['SOURCE_CABIN_IND']] = 'Y'
    
    # If they are in Tier 3, do they have the status for Tier 3 access, either BA or OneWorld?
    
    tier_3_status_list = ['Silver','SAPP']
    tier_3_status_y = (df['Lounge_eligibility_tier']=='Tier 3') & ((df['BA_PAX_TIER'].isin(tier_3_status_list)|(df['ONEWORLD_TIER'].isin(tier_3_status_list))))
    df.loc[tier_3_status_y,['SOURCE_STATUS_IND']] = 'Y'
    
    return df

In [62]:
# run this function.
df_final = tier_source_finder(df_final)

Add haul data

Hauls vary by both location and by date. Some routes have switched haul relatively recently e.g. Amman, Cairo.
Some long haul routes are shorter than short-haul routes e.g. Tel Aviv is shorter than Amman.

A valid way of categorising flights is by selling classes: only long haul flights sell First (F) and World Traveller Plus (W). Short haul flights sell Club as C and long haul flights sell Club as J.


In [63]:
# First, group flights by Skew ID and passengers in each cabin per Skew ID. Fill missing values with zero.

def add_haul(df):

    grouped_df_destinations_dates = df.groupby(['Skew_Id','TRAVEL_CABIN_CD']).agg({'pax':'sum'}).reset_index()
    pivot_destination_dates = grouped_df_destinations_dates.pivot(index='Skew_Id',columns='TRAVEL_CABIN_CD',values='pax').reset_index()
    pivot_destination_dates.fillna(0,inplace=True)
    pivot_destination_dates['HAUL'] = pd.NaT

    # work out if long or short haul. The data are clean, there are none selling in both C and any of F, J and W.
    long_haul_ind = (pivot_destination_dates['F']>0)|(pivot_destination_dates['J']>0)|(pivot_destination_dates['W']>0) # any number of seats sold in First, Club World or WTP.
    short_haul_ind = (pivot_destination_dates['C']>0) # any number of seats sold in Club Europe means flight is short haul.
    
    pivot_destination_dates.loc[long_haul_ind,['HAUL']] = 'L'
    pivot_destination_dates.loc[short_haul_ind,['HAUL']] = 'S'
    
    df_merged = pd.merge(df,pivot_destination_dates[['Skew_Id','HAUL']],on='Skew_Id')
    
    return df_merged

df_final = add_haul(df_final)

# There are 501 missing values for Haul, but we can ignore these given total dataset contains 131,162 flights.

In [64]:
df_final.head(2)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,STATUS,SOURCE_CABIN_IND,SOURCE_STATUS_IND,HAUL
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,2023,Emerald,N,Y,S
1,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold Guest List,,Tier 1,1,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,2023,Emerald,N,Y,S


In [65]:
# TEMP

# EXPORT TO CSV (VERY BIG)
# CSV needs to be moved after production as it is too large to be pushed to github.
create_csv = False
if create_csv:
    df_final.to_csv('df_final.csv')

### Create Pivot Table

In [66]:
fields_to_keep = ['Skew_Id',
                  'OPERATING_FLT_NO',
                  'GMT_UPLIFT_DT',
                  'GMT_PLND_DEP_TIME',
                  'ACT_DEP_TML_CD',
                  'DISCHARGE_STN_CD',
                  'IATA_AC_TYP_CD',
                  'ACT_AC_TYP_CD',
                  'COUNTRY_CD',
                  'COUNTRY_NM',
                  'CORP_GEOG_CTRY_GRP_NM',
                  'CORP_GEOG_CONTINENT_NM',
                  'WB_NB_CAT',
                  'FIRST_SEATS_QTY',
                  'CLUB_SEATS_QTY',
                  'PREM_ECONOMY_SEATS_QTY',
                  'ECONOMY_SEATS_QTY',
                  'GMT_UPLIFT_YR',
                  'HAUL']

df_to_pivot = df_final.copy()

print(fields_to_keep)

pivot_df = pd.pivot_table(df_to_pivot, values = 'pax', index = fields_to_keep,
                          columns = 'Lounge_eligibility_tier', aggfunc = 'sum', fill_value = 0).reset_index()

pivot_df['Total_Passengers'] = pivot_df['Not eligible'] + pivot_df['Tier 1'] + pivot_df['Tier 2'] + pivot_df['Tier 3']
pivot_df['Not eligible ratio'] = pivot_df['Not eligible']/pivot_df['Total_Passengers']
pivot_df['Tier 1 ratio'] = pivot_df['Tier 1']/pivot_df['Total_Passengers']
pivot_df['Tier 2 ratio'] = pivot_df['Tier 2']/pivot_df['Total_Passengers']
pivot_df['Tier 3 ratio'] = pivot_df['Tier 3']/pivot_df['Total_Passengers']

print(pivot_df.shape)
pivot_df.head(10)

['Skew_Id', 'OPERATING_FLT_NO', 'GMT_UPLIFT_DT', 'GMT_PLND_DEP_TIME', 'ACT_DEP_TML_CD', 'DISCHARGE_STN_CD', 'IATA_AC_TYP_CD', 'ACT_AC_TYP_CD', 'COUNTRY_CD', 'COUNTRY_NM', 'CORP_GEOG_CTRY_GRP_NM', 'CORP_GEOG_CONTINENT_NM', 'WB_NB_CAT', 'FIRST_SEATS_QTY', 'CLUB_SEATS_QTY', 'PREM_ECONOMY_SEATS_QTY', 'ECONOMY_SEATS_QTY', 'GMT_UPLIFT_YR', 'HAUL']
(130785, 28)


Lounge_eligibility_tier,Skew_Id,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TIME,ACT_DEP_TML_CD,DISCHARGE_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,HAUL,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passengers,Not eligible ratio,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio
0,2019-03-31103,103,2019-03-31,17:30:00,5,YYC,788,B8,CA,Canada,CANADA,NORTH AMERICA,WB,0.0,35.0,25.0,154.0,2019,L,154,1,9,33,197,0.781726,0.005076,0.045685,0.167513
1,2019-03-31105,105,2019-03-31,21:25:00,5,DXB,777,V7,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,L,137,13,13,46,209,0.655502,0.062201,0.062201,0.220096
2,2019-03-31107,107,2019-03-31,11:50:00,5,DXB,777,V7,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,L,126,11,18,46,201,0.626866,0.054726,0.089552,0.228856
3,2019-03-31109,109,2019-03-31,20:15:00,5,DXB,777,V7,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,L,139,14,22,42,217,0.640553,0.064516,0.101382,0.193548
4,2019-03-3111,11,2019-03-31,18:55:00,5,SIN,388,A8,SG,Singapore,FAR EAST JSA HUBS,AUSTRALASIA PACIFIC,WB,14.0,97.0,55.0,303.0,2019,L,308,17,43,75,443,0.695260,0.038375,0.097065,0.169300
5,2019-03-31113,113,2019-03-31,15:15:00,5,JFK,777,W7,US,United States,USA,NORTH AMERICA,WB,12.0,48.0,32.0,127.0,2019,L,135,12,22,43,212,0.636792,0.056604,0.103774,0.202830
6,2019-03-31115,115,2019-03-31,13:30:00,5,JFK,777,V7,US,United States,USA,NORTH AMERICA,WB,14.0,48.0,40.0,124.0,2019,L,132,14,26,40,212,0.622642,0.066038,0.122642,0.188679
7,2019-03-31117,117,2019-03-31,07:30:00,5,JFK,744,S4,US,United States,USA,NORTH AMERICA,WB,14.0,86.0,30.0,145.0,2019,L,138,19,16,61,234,0.589744,0.081197,0.068376,0.260684
8,2019-03-31119,119,2019-03-31,13:15:00,5,BLR,77W,G7,IN,India,SOUTH ASIA,ASIA SOUTH EAST AND CENTRAL,WB,14.0,56.0,44.0,183.0,2019,L,204,3,12,42,261,0.781609,0.011494,0.045977,0.160920
9,2019-03-31123,123,2019-03-31,18:45:00,5,DOH,789,L8,QA,Qatar,SOUTHERN GULF,MIDDLE EAST,WB,8.0,42.0,39.0,127.0,2019,L,133,6,11,27,177,0.751412,0.033898,0.062147,0.152542


In [67]:
df_to_pivot = df_final.copy()

pivot_df_additional = pd.pivot_table(df_to_pivot, values = 'pax', index = "Skew_Id",
                          columns = 'BOOKED_CABIN_CD', aggfunc = 'sum', fill_value = 0).reset_index()

print(pivot_df_additional.shape)
pivot_df_additional.head(5)

(131362, 6)


BOOKED_CABIN_CD,Skew_Id,C,F,J,M,W
0,2019-03-31103,0,0,29,143,25
1,2019-03-31105,0,8,44,121,36
2,2019-03-31107,0,4,48,117,32
3,2019-03-31109,0,7,48,124,38
4,2019-03-3111,0,13,96,280,54


In [68]:
pivot_df_merged = pd.merge(left = pivot_df, left_on = 'Skew_Id',
                           right = pivot_df_additional, right_on = 'Skew_Id',
                           how = 'left')

print(pivot_df_merged.shape)
pivot_df_merged.head(10)

(130785, 33)


,Skew_Id,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TIME,ACT_DEP_TML_CD,DISCHARGE_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,HAUL,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passengers,Not eligible ratio,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio,C,F,J,M,W
0,2019-03-31103,103,2019-03-31,17:30:00,5,YYC,788,B8,CA,Canada,CANADA,NORTH AMERICA,WB,0.0,35.0,25.0,154.0,2019,L,154,1,9,33,197,0.781726,0.005076,0.045685,0.167513,0,0,29,143,25
1,2019-03-31105,105,2019-03-31,21:25:00,5,DXB,777,V7,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,L,137,13,13,46,209,0.655502,0.062201,0.062201,0.220096,0,8,44,121,36
2,2019-03-31107,107,2019-03-31,11:50:00,5,DXB,777,V7,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,L,126,11,18,46,201,0.626866,0.054726,0.089552,0.228856,0,4,48,117,32
3,2019-03-31109,109,2019-03-31,20:15:00,5,DXB,777,V7,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,L,139,14,22,42,217,0.640553,0.064516,0.101382,0.193548,0,7,48,124,38
4,2019-03-3111,11,2019-03-31,18:55:00,5,SIN,388,A8,SG,Singapore,FAR EAST JSA HUBS,AUSTRALASIA PACIFIC,WB,14.0,97.0,55.0,303.0,2019,L,308,17,43,75,443,0.695260,0.038375,0.097065,0.169300,0,13,96,280,54
5,2019-03-31113,113,2019-03-31,15:15:00,5,JFK,777,W7,US,United States,USA,NORTH AMERICA,WB,12.0,48.0,32.0,127.0,2019,L,135,12,22,43,212,0.636792,0.056604,0.103774,0.202830,0,10,40,128,34
6,2019-03-31115,115,2019-03-31,13:30:00,5,JFK,777,V7,US,United States,USA,NORTH AMERICA,WB,14.0,48.0,40.0,124.0,2019,L,132,14,26,40,212,0.622642,0.066038,0.122642,0.188679,0,12,50,112,38
7,2019-03-31117,117,2019-03-31,07:30:00,5,JFK,744,S4,US,United States,USA,NORTH AMERICA,WB,14.0,86.0,30.0,145.0,2019,L,138,19,16,61,234,0.589744,0.081197,0.068376,0.260684,0,14,64,135,21
8,2019-03-31119,119,2019-03-31,13:15:00,5,BLR,77W,G7,IN,India,SOUTH ASIA,ASIA SOUTH EAST AND CENTRAL,WB,14.0,56.0,44.0,183.0,2019,L,204,3,12,42,261,0.781609,0.011494,0.045977,0.160920,0,0,45,189,27
9,2019-03-31123,123,2019-03-31,18:45:00,5,DOH,789,L8,QA,Qatar,SOUTHERN GULF,MIDDLE EAST,WB,8.0,42.0,39.0,127.0,2019,L,133,6,11,27,177,0.751412,0.033898,0.062147,0.152542,0,4,28,137,8


### Group by functions

Very useful for later

In [69]:
def group_pivot(df, list_groupby):
    Tiers = ['Not_Eligible','Tier_1','Tier_2','Tier_3']

    df_grouped = df.groupby(list_groupby).agg(
        Not_Eligible = ('Not eligible', sum),
        Tier_1 = ('Tier 1', sum),
        Tier_2 = ('Tier 2', sum),
        Tier_3 = ('Tier 3', sum),
        TOTAL_PAX = ('Total_Passengers', sum)).reset_index()

    for t in Tiers:
        df_grouped[t] = (df_grouped[t] / df_grouped['TOTAL_PAX']) * 100
        df_grouped[t] = df_grouped[t].map('{:,.2f}%'.format)

    return df_grouped

def group_pivot_to_lookup(df):
    return df.set_index(list(df.columns))

In [70]:
pivot_df_merged.head(1)

,Skew_Id,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TIME,ACT_DEP_TML_CD,DISCHARGE_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,HAUL,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passengers,Not eligible ratio,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio,C,F,J,M,W
0,2019-03-31103,103,2019-03-31,17:30:00,5,YYC,788,B8,CA,Canada,CANADA,NORTH AMERICA,WB,0.0,35.0,25.0,154.0,2019,L,154,1,9,33,197,0.781726,0.005076,0.045685,0.167513,0,0,29,143,25


### Analysis

### Terminal differences

We have also decided to split by terminals to see how the eligibilities has changed for each terminals as well.

Here we can see the eligibililies are more skewed towards to Termnal 5 eligibilities due to T5 serving more passengers than T5 (5.46 x in 2019, 9.77 x in 2023). We can see there is a a significant drop for the Tier 1 elgibility in T3, from 1.55% to 0.41% (-1.14%, $\frac{2}{3}$ of 2019).

Splitting by both year and by terminal has shows us that there are fewer business travellers in 2023 than in 2019

In [71]:
pivot_df_merged_group_by_term = group_pivot(pivot_df_merged, ['ACT_DEP_TML_CD','GMT_UPLIFT_YR'])
pivot_df_merged_group_by_term

,ACT_DEP_TML_CD,GMT_UPLIFT_YR,Not_Eligible,Tier_1,Tier_2,Tier_3,TOTAL_PAX
0,3,2019,77.45%,1.55%,4.75%,16.24%,1687694
1,3,2023,79.40%,0.41%,4.64%,15.55%,902287
2,5,2019,73.52%,1.84%,6.01%,18.63%,9234462
3,5,2023,75.33%,1.02%,5.20%,18.45%,8828814


### Group by Year

We have decided to split the flights by year to see how the eligibility has been changed from 2019 to 2023. Here we have summed up the passengers of each flight by year and divided by the total number of passengers.

We can see that the proportion of customers not eligibile for any lounge has been increased by 1.57%pt, as a result of the drop of percentages of all lounge eligibility. The biggest drop happens on Tier 1, where it dropped from 1.80% to 0.96%, down by nearly 50%. The eligibility for Tier 3 being the smallest change: form 18.25% to 18.18% (-0.38%).

In [72]:
df_pivot_groupby_year = group_pivot(pivot_df_merged, ['GMT_UPLIFT_YR'])
df_pivot_groupby_year

,GMT_UPLIFT_YR,Not_Eligible,Tier_1,Tier_2,Tier_3,TOTAL_PAX
0,2019,74.13%,1.80%,5.81%,18.26%,10922156
1,2023,75.71%,0.96%,5.15%,18.19%,9731101


### Group by Country Group and Year

In [73]:
df_pivot_groupby_country_group_year = group_pivot(pivot_df_merged, ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR'])
group_pivot_to_lookup(df_pivot_groupby_country_group_year)
# df_pivot_groupby_country_group_year.head(10)

Empty DataFrame
Columns: []
Index: [(AFRICA, EAST AND CENTRAL AFRICA, 2019, 74.77%, 3.65%, 4.01%, 17.57%, 67571), (AFRICA, EAST AND CENTRAL AFRICA, 2023, 79.40%, 0.24%, 4.32%, 16.04%, 51668), (AFRICA, NORTH AFRICA, 2019, 73.72%, 0.47%, 4.95%, 20.86%, 41975), (AFRICA, NORTH AFRICA, 2023, 76.70%, 0.26%, 3.62%, 19.43%, 94026), (AFRICA, SOUTHERN AFRICA, 2019, 70.66%, 3.78%, 5.33%, 20.22%, 220175), (AFRICA, SOUTHERN AFRICA, 2023, 74.43%, 2.41%, 4.44%, 18.72%, 201263), (AFRICA, WEST AFRICA, 2019, 75.77%, 3.73%, 4.48%, 16.02%, 158447), (AFRICA, WEST AFRICA, 2023, 75.95%, 2.34%, 4.51%, 17.20%, 130417), (ASIA FAR EAST, CHINA, 2019, 72.37%, 4.63%, 4.23%, 18.77%, 105600), (ASIA FAR EAST, CHINA, 2023, 77.80%, 0.25%, 2.58%, 19.37%, 48671), (ASIA FAR EAST, FAR EAST HUBS, 2019, 72.29%, 3.84%, 5.35%, 18.52%, 174262), (ASIA FAR EAST, FAR EAST HUBS, 2023, 78.92%, 1.44%, 3.43%, 16.21%, 93144), (ASIA FAR EAST, FAR EAST OTHER, 2019, 80.35%, 0.36%, 3.79%, 15.50%, 39313), (ASIA FAR EAST, JAPAN, 2019, 73.10%, 3.53%, 4.79%, 18.57%, 111747), (ASIA FAR EAST, JAPAN, 2023, 73.78%, 1.02%, 4.39%, 20.81%, 54625), (ASIA SOUTH EAST AND CENTRAL, SOUTH ASIA, 2019, 74.03%, 2.84%, 4.05%, 19.09%, 286689), (ASIA SOUTH EAST AND CENTRAL, SOUTH ASIA, 2023, 79.55%, 0.43%, 3.02%, 17.01%, 298669), (AUSTRALASIA PACIFIC, FAR EAST JSA HUBS, 2019, 73.86%, 2.28%, 5.56%, 18.30%, 130531), (AUSTRALASIA PACIFIC, FAR EAST JSA HUBS, 2023, 62.91%, 3.44%, 7.37%, 26.28%, 43047), (EUROPE EXC UK, BENELUX, 2019, 72.65%, 0.74%, 8.32%, 18.29%, 374814), (EUROPE EXC UK, BENELUX, 2023, 77.68%, 0.50%, 6.34%, 15.47%, 288010), (EUROPE EXC UK, EASTERN EUROPE, 2019, 80.22%, 0.50%, 4.83%, 14.45%, 613530), (EUROPE EXC UK, EASTERN EUROPE, 2023, 80.19%, 0.24%, 4.22%, 15.35%, 459852), (EUROPE EXC UK, IRELAND RO, 2019, 75.53%, 0.60%, 6.64%, 17.23%, 202205), (EUROPE EXC UK, IRELAND RO, 2023, 75.84%, 0.64%, 6.34%, 17.19%, 126562), (EUROPE EXC UK, MEDITERRANEAN, 2019, 75.01%, 0.72%, 5.76%, 18.51%, 1210567), (EUROPE EXC UK, MEDITERRANEAN, 2023, 73.70%, 0.30%, 5.28%, 20.71%, 1418790), (EUROPE EXC UK, SCANDINAVIA, 2019, 76.69%, 0.57%, 6.70%, 16.04%, 530327), (EUROPE EXC UK, SCANDINAVIA, 2023, 77.04%, 0.39%, 6.03%, 16.54%, 375142), (EUROPE EXC UK, WEST EUROPE, 2019, 75.42%, 0.51%, 6.80%, 17.27%, 2610448), (EUROPE EXC UK, WEST EUROPE, 2023, 76.26%, 0.38%, 5.74%, 17.61%, 2325062), (MIDDLE EAST, MIDDLE EAST OTHER, 2019, 80.19%, 0.34%, 4.69%, 14.77%, 52779), (MIDDLE EAST, MIDDLE EAST OTHER, 2023, 78.34%, 0.14%, 3.62%, 17.90%, 22495), (MIDDLE EAST, NORTHERN GULF, 2019, 66.83%, 5.01%, 4.80%, 23.36%, 63033), (MIDDLE EAST, NORTHERN GULF, 2023, 76.24%, 3.03%, 2.96%, 17.78%, 53351), (MIDDLE EAST, SAUDI ARABIA, 2019, 65.71%, 4.03%, 6.63%, 23.63%, 65967), (MIDDLE EAST, SAUDI ARABIA, 2023, 67.20%, 3.51%, 7.76%, 21.53%, 31350), (MIDDLE EAST, SOUTHERN GULF, 2019, 69.43%, 4.78%, 6.58%, 19.22%, 154064), (MIDDLE EAST, SOUTHERN GULF, 2023, 72.14%, 3.62%, 5.50%, 18.73%, 112484), (NORTH AMERICA, CANADA, 2019, 76.61%, 2.34%, 3.54%, 17.52%, 247530), (NORTH AMERICA, CANADA, 2023, 79.10%, 0.23%, 3.40%, 17.28%, 191259), (NORTH AMERICA, NORTH AMERICA OTHER, 2023, 65.73%, 3.60%, 10.58%, 20.09%, 34024), (NORTH AMERICA, USA, 2019, 69.43%, 4.36%, 5.38%, 20.84%, 2109416), (NORTH AMERICA, USA, 2023, 72.97%, 2.44%, 5.19%, 19.40%, 1917136), (SOUTH AMERICA INC CARIBBEAN, CARIBBEAN, 2019, 72.58%, 0.00%, 7.26%, 20.16%, 124), (SOUTH AMERICA INC CARIBBEAN, CARIBBEAN, 2023, 73.50%, 2.66%, 3.67%, 20.17%, 39308), (SOUTH AMERICA INC CARIBBEAN, CENTRAL AMERICA, 2019, 73.91%, 3.92%, 4.25%, 17.93%, 35254), (SOUTH AMERICA INC CARIBBEAN, CENTRAL AMERICA, 2023, 76.70%, 3.56%, 2.54%, 17.20%, 33663), (SOUTH AMERICA INC CARIBBEAN, SOUTH AMERICA, 2019, 77.09%, 2.13%, 3.45%, 17.32%, 164725), (SOUTH AMERICA INC CARIBBEAN, SOUTH AMERICA, 2023, 80.05%, 0.58%, 3.01%, 16.36%, 123993), (UK, UK, 2019, 75.77%, 0.42%, 5.68%, 18.14%, 1151063), (UK, UK, 2023, 77.83%, 0.36%, 5.00%, 16.81%, 1163090)]

### Group by Continent

We have also decided to split the flights by year and continent to see how eligibility is affected by the location of the destination.

In 2019, Africa, Asia (both Far East & South East and Central), Middle East and North America had more than 2.5% of passengers eligible for the Tier 1 lounge, while the Europe and UK has less that 0.6%.

In 2023, Australasia Pacific is the only continent that recorded an increase on all eligibilities, while the rest has recorded a decreased for all tiers. Asia shown the largest decrease, espicially for Tier 1, 3.64% to 1.03% for Far East and 2.84% to 0.43% for SE and Central Asia.

In [74]:
df_pivot_groupby_cont_year = group_pivot(pivot_df_merged, ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR'])

group_pivot_to_lookup(df_pivot_groupby_cont_year)

Empty DataFrame
Columns: []
Index: [(AFRICA, 2019, 73.15%, 3.46%, 4.84%, 18.55%, 488168), (AFRICA, 2023, 75.83%, 1.73%, 4.28%, 18.15%, 477374), (ASIA FAR EAST, 2019, 73.26%, 3.64%, 4.79%, 18.32%, 430922), (ASIA FAR EAST, 2023, 77.21%, 1.03%, 3.49%, 18.27%, 196440), (ASIA SOUTH EAST AND CENTRAL, 2019, 74.03%, 2.84%, 4.05%, 19.09%, 286689), (ASIA SOUTH EAST AND CENTRAL, 2023, 79.55%, 0.43%, 3.02%, 17.01%, 298669), (AUSTRALASIA PACIFIC, 2019, 73.86%, 2.28%, 5.56%, 18.30%, 130531), (AUSTRALASIA PACIFIC, 2023, 62.91%, 3.44%, 7.37%, 26.28%, 43047), (EUROPE EXC UK, 2019, 75.80%, 0.58%, 6.44%, 17.18%, 5541891), (EUROPE EXC UK, 2023, 76.03%, 0.36%, 5.54%, 18.07%, 4993418), (MIDDLE EAST, 2019, 69.90%, 3.98%, 5.96%, 20.16%, 335843), (MIDDLE EAST, 2023, 73.07%, 3.11%, 5.01%, 18.82%, 219680), (NORTH AMERICA, 2019, 70.18%, 4.15%, 5.19%, 20.49%, 2356946), (NORTH AMERICA, 2023, 73.40%, 2.26%, 5.11%, 19.22%, 2142419), (SOUTH AMERICA INC CARIBBEAN, 2019, 76.53%, 2.45%, 3.59%, 17.43%, 200103), (SOUTH AMERICA INC CARIBBEAN, 2023, 78.17%, 1.51%, 3.06%, 17.26%, 196964), (UK, 2019, 75.77%, 0.42%, 5.68%, 18.14%, 1151063), (UK, 2023, 77.83%, 0.36%, 5.00%, 16.81%, 1163090)]

### Group by Aircraft

We also split the flights by aircraft to see how the number of premium seats affect eligibility

We see there is a strong positive correlation between Tier 1 eligibility and the number of First seats on an aircraft.

Hoever, we are not able to see any strong correlation between the number of Club seats and any tier eligibilities.


In [75]:
df_pivot_groupby_acft = group_pivot(pivot_df_merged, ['IATA_AC_TYP_CD','ACT_AC_TYP_CD','FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY'])
group_pivot_to_lookup(df_pivot_groupby_acft)

Empty DataFrame
Columns: []
Index: [(319, A4, 0.0, 16.0, 0.0, 119.0, 74.91%, 0.52%, 6.65%, 17.91%, 2541068), (319, M4, 0.0, 21.0, 0.0, 113.0, 79.03%, 0.28%, 4.87%, 15.82%, 433739), (320, A3, 0.0, 24.0, 0.0, 132.0, 76.76%, 0.41%, 5.91%, 16.91%, 1989201), (320, E9, 0.0, 29.0, 0.0, 130.0, 75.33%, 0.30%, 5.01%, 19.37%, 198991), (320, M2, 0.0, 24.0, 0.0, 144.0, 80.32%, 0.27%, 4.38%, 15.04%, 50257), (320, R3, 0.0, 28.0, 0.0, 139.0, 76.24%, 0.37%, 5.57%, 17.81%, 2357986), (320, T3, 0.0, 26.0, 0.0, 142.0, 79.32%, 0.27%, 4.34%, 16.06%, 168327), (321, M6, 0.0, 23.0, 0.0, 131.0, 79.23%, 0.59%, 5.74%, 14.44%, 136777), (321, V6, 0.0, 38.0, 0.0, 160.0, 76.69%, 0.43%, 5.95%, 16.92%, 1013570), (321, W6, 0.0, 42.0, 0.0, 154.0, 77.34%, 0.44%, 5.67%, 16.56%, 204879), (32A, H3, 0.0, 20.0, 0.0, 150.0, 75.87%, 0.42%, 5.89%, 17.82%, 1085972), (32N, N3, 0.0, 20.0, 0.0, 150.0, 75.77%, 0.39%, 5.68%, 18.17%, 1700471), (32Q, N6, 0.0, 32.0, 0.0, 172.0, 76.59%, 0.36%, 5.15%, 17.90%, 889488), (332, W9, 0.0, 30.0, 21.0, 235.0, 84.89%, 0.38%, 4.97%, 9.75%, 523), (339, W9, 0.0, 30.0, 21.0, 234.0, 84.56%, 0.18%, 3.69%, 11.57%, 23093), (343, W9, 0.0, 45.0, 0.0, 212.0, 80.09%, 0.33%, 3.87%, 15.72%, 43925), (351, K5, 0.0, 56.0, 56.0, 219.0, 79.38%, 0.26%, 4.10%, 16.26%, 582982), (388, A8, 14.0, 97.0, 55.0, 303.0, 72.40%, 3.20%, 5.08%, 19.33%, 1243136), (744, S4, 14.0, 86.0, 30.0, 145.0, 60.85%, 5.19%, 7.03%, 26.92%, 675347), (744, V4, 14.0, 52.0, 36.0, 235.0, 76.80%, 3.79%, 3.83%, 15.58%, 715726), (777, D7, 0.0, 48.0, 40.0, 184.0, 78.42%, 0.25%, 3.97%, 17.37%, 402321), (777, J7, 0.0, 32.0, 48.0, 252.0, 74.31%, 0.00%, 5.56%, 20.14%, 144), (777, K7, 0.0, 48.0, 24.0, 203.0, 78.21%, 0.00%, 3.85%, 17.95%, 156), (777, L7, 0.0, 48.0, 24.0, 203.0, 78.30%, 0.27%, 4.38%, 17.05%, 155095), (777, M7, 7.0, 50.0, 40.0, 138.0, 71.40%, 3.33%, 5.93%, 19.34%, 450915), (777, P7, 17.0, 48.0, 24.0, 127.0, 65.77%, 6.51%, 5.97%, 21.75%, 108147), (777, R7, 10.0, 52.0, 40.0, 134.0, 69.66%, 3.99%, 6.81%, 19.54%, 45011), (777, V7, 14.0, 48.0, 40.0, 124.0, 68.16%, 5.21%, 5.92%, 20.71%, 538568), (777, W7, 12.0, 48.0, 32.0, 127.0, 68.59%, 5.15%, 5.11%, 21.16%, 270777), (77W, F7, 8.0, 76.0, 40.0, 130.0, 63.94%, 3.36%, 6.84%, 25.86%, 374664), (77W, G7, 14.0, 56.0, 44.0, 183.0, 71.83%, 4.40%, 4.99%, 18.77%, 525688), (781, X8, 7.0, 49.0, 35.0, 165.0, 74.74%, 2.78%, 4.31%, 18.16%, 243361), (788, B8, 0.0, 35.0, 25.0, 154.0, 80.93%, 0.19%, 3.22%, 15.66%, 647096), (789, L8, 8.0, 42.0, 39.0, 127.0, 73.59%, 2.65%, 4.64%, 19.11%, 835856)]

In [76]:
df_pivot_groupby_First_Cont_Acft_typ = group_pivot(pivot_df_merged, ['FIRST_SEATS_QTY','CORP_GEOG_CONTINENT_NM','WB_NB_CAT'])
group_pivot_to_lookup(df_pivot_groupby_First_Cont_Acft_typ)


Empty DataFrame
Columns: []
Index: [(0.0, AFRICA, NB, 76.19%, 0.28%, 3.76%, 19.77%, 100414), (0.0, AFRICA, WB, 79.61%, 0.37%, 4.28%, 15.73%, 188854), (0.0, ASIA FAR EAST, WB, 80.97%, 0.24%, 3.31%, 15.47%, 127412), (0.0, ASIA SOUTH EAST AND CENTRAL, WB, 80.11%, 0.21%, 3.11%, 16.57%, 362863), (0.0, AUSTRALASIA PACIFIC, WB, 79.73%, 0.29%, 4.16%, 15.82%, 49955), (0.0, EUROPE EXC UK, NB, 76.05%, 0.43%, 5.97%, 17.55%, 10281017), (0.0, EUROPE EXC UK, WB, 76.80%, 0.57%, 5.97%, 16.66%, 66335), (0.0, MIDDLE EAST, NB, 79.64%, 0.28%, 4.37%, 15.71%, 75274), (0.0, MIDDLE EAST, WB, 74.01%, 0.69%, 7.21%, 18.09%, 20590), (0.0, NORTH AMERICA, WB, 79.52%, 0.18%, 3.86%, 16.44%, 875715), (0.0, SOUTH AMERICA INC CARIBBEAN, WB, 80.81%, 0.19%, 3.16%, 15.84%, 163611), (0.0, UK, NB, 76.81%, 0.39%, 5.34%, 17.47%, 2314021), (7.0, AFRICA, WB, 72.42%, 3.10%, 4.82%, 19.66%, 40046), (7.0, EUROPE EXC UK, WB, 73.95%, 0.48%, 8.01%, 17.56%, 2933), (7.0, MIDDLE EAST, WB, 72.23%, 3.32%, 5.23%, 19.22%, 122430), (7.0, NORTH AMERICA, WB, 72.57%, 3.16%, 5.55%, 18.73%, 492566), (7.0, SOUTH AMERICA INC CARIBBEAN, WB, 73.79%, 2.56%, 3.66%, 19.99%, 36301), (8.0, AFRICA, WB, 69.53%, 3.12%, 5.35%, 22.01%, 37328), (8.0, ASIA FAR EAST, WB, 73.96%, 2.28%, 4.15%, 19.61%, 202802), (8.0, ASIA SOUTH EAST AND CENTRAL, WB, 73.49%, 2.69%, 3.95%, 19.87%, 103372), (8.0, AUSTRALASIA PACIFIC, WB, 62.78%, 3.36%, 7.44%, 26.42%, 41164), (8.0, EUROPE EXC UK, WB, 67.59%, 1.88%, 10.31%, 20.22%, 47976), (8.0, MIDDLE EAST, WB, 72.88%, 3.64%, 5.36%, 18.12%, 96546), (8.0, NORTH AMERICA, WB, 68.01%, 3.01%, 5.89%, 23.10%, 543312), (8.0, SOUTH AMERICA INC CARIBBEAN, WB, 75.84%, 2.93%, 3.44%, 17.79%, 138020), (10.0, AFRICA, WB, 70.60%, 4.40%, 5.06%, 19.94%, 1956), (10.0, EUROPE EXC UK, WB, 71.32%, 1.03%, 7.58%, 20.07%, 1161), (10.0, MIDDLE EAST, WB, 74.42%, 4.48%, 3.59%, 17.50%, 12109), (10.0, NORTH AMERICA, WB, 67.52%, 3.85%, 8.29%, 20.33%, 29172), (10.0, SOUTH AMERICA INC CARIBBEAN, WB, 71.13%, 4.73%, 4.08%, 20.07%, 613), (12.0, AFRICA, WB, 71.93%, 4.12%, 4.16%, 19.79%, 2451), (12.0, ASIA FAR EAST, WB, 70.32%, 4.86%, 4.96%, 19.86%, 53523), (12.0, ASIA SOUTH EAST AND CENTRAL, WB, 69.18%, 5.26%, 4.41%, 21.15%, 97807), (12.0, AUSTRALASIA PACIFIC, WB, 72.28%, 0.00%, 3.22%, 24.50%, 404), (12.0, EUROPE EXC UK, WB, 68.51%, 3.63%, 7.48%, 20.37%, 4516), (12.0, MIDDLE EAST, WB, 67.21%, 4.86%, 4.58%, 23.35%, 11882), (12.0, NORTH AMERICA, WB, 67.14%, 5.36%, 5.86%, 21.65%, 99853), (12.0, SOUTH AMERICA INC CARIBBEAN, WB, 71.55%, 0.00%, 4.40%, 24.05%, 341), (14.0, AFRICA, WB, 73.06%, 3.63%, 4.72%, 18.58%, 589392), (14.0, ASIA FAR EAST, WB, 72.47%, 4.17%, 5.00%, 18.36%, 243625), (14.0, ASIA SOUTH EAST AND CENTRAL, WB, 72.69%, 3.44%, 4.42%, 19.45%, 21316), (14.0, AUSTRALASIA PACIFIC, WB, 70.10%, 3.58%, 6.44%, 19.88%, 82055), (14.0, EUROPE EXC UK, WB, 67.84%, 3.52%, 7.77%, 20.87%, 123979), (14.0, MIDDLE EAST, WB, 66.66%, 4.98%, 6.38%, 21.98%, 191192), (14.0, NORTH AMERICA, WB, 69.95%, 4.25%, 5.28%, 20.52%, 2388593), (14.0, SOUTH AMERICA INC CARIBBEAN, WB, 73.46%, 4.37%, 3.34%, 18.83%, 58181), (14.0, UK, WB, 69.70%, 0.76%, 6.06%, 23.48%, 132), (17.0, AFRICA, WB, 69.79%, 3.10%, 4.63%, 22.49%, 5101), (17.0, EUROPE EXC UK, WB, 65.30%, 5.70%, 6.97%, 22.04%, 7392), (17.0, MIDDLE EAST, WB, 66.05%, 6.33%, 5.83%, 21.78%, 25500), (17.0, NORTH AMERICA, WB, 65.43%, 6.91%, 6.01%, 21.65%, 70154)]

Take a closer look into European market

In [77]:
pivot_df_merged_EU = pivot_df_merged[pivot_df_merged['CORP_GEOG_CONTINENT_NM'] == 'EUROPE EXC UK']

df_pivot_groupby_First_EU_Coun_Acft_typ = group_pivot(pivot_df_merged_EU, ['GMT_UPLIFT_YR','FIRST_SEATS_QTY','CORP_GEOG_CTRY_GRP_NM','WB_NB_CAT'])
group_pivot_to_lookup(df_pivot_groupby_First_EU_Coun_Acft_typ)

Empty DataFrame
Columns: []
Index: [(2019, 0.0, BENELUX, NB, 72.65%, 0.74%, 8.32%, 18.29%, 374814), (2019, 0.0, EASTERN EUROPE, NB, 80.78%, 0.33%, 4.62%, 14.27%, 580285), (2019, 0.0, IRELAND RO, NB, 75.53%, 0.60%, 6.64%, 17.23%, 202205), (2019, 0.0, MEDITERRANEAN, NB, 75.71%, 0.39%, 5.57%, 18.32%, 1117588), (2019, 0.0, MEDITERRANEAN, WB, 73.76%, 0.62%, 6.96%, 18.66%, 12982), (2019, 0.0, SCANDINAVIA, NB, 76.69%, 0.57%, 6.70%, 16.04%, 530327), (2019, 0.0, WEST EUROPE, NB, 75.50%, 0.51%, 6.80%, 17.19%, 2564697), (2019, 0.0, WEST EUROPE, WB, 73.70%, 0.78%, 7.06%, 18.46%, 7026), (2019, 8.0, EASTERN EUROPE, WB, 70.00%, 3.29%, 9.36%, 17.35%, 18341), (2019, 8.0, WEST EUROPE, WB, 71.88%, 0.52%, 8.33%, 19.27%, 192), (2019, 12.0, EASTERN EUROPE, WB, 73.90%, 2.26%, 8.58%, 15.26%, 839), (2019, 12.0, MEDITERRANEAN, WB, 66.19%, 4.47%, 7.44%, 21.90%, 3174), (2019, 12.0, WEST EUROPE, WB, 74.16%, 0.60%, 5.96%, 19.28%, 503), (2019, 14.0, EASTERN EUROPE, WB, 71.21%, 3.49%, 7.13%, 18.16%, 13830), (2019, 14.0, MEDITERRANEAN, WB, 65.48%, 5.20%, 8.36%, 20.96%, 71030), (2019, 14.0, WEST EUROPE, WB, 70.50%, 0.49%, 7.09%, 21.91%, 36666), (2019, 17.0, EASTERN EUROPE, WB, 65.11%, 6.38%, 11.49%, 17.02%, 235), (2019, 17.0, MEDITERRANEAN, WB, 64.30%, 6.97%, 6.92%, 21.80%, 5793), (2019, 17.0, WEST EUROPE, WB, 69.57%, 0.15%, 6.38%, 23.90%, 1364), (2023, 0.0, BENELUX, NB, 77.68%, 0.50%, 6.34%, 15.47%, 288010), (2023, 0.0, EASTERN EUROPE, NB, 80.19%, 0.24%, 4.22%, 15.35%, 459852), (2023, 0.0, IRELAND RO, NB, 75.84%, 0.64%, 6.34%, 17.19%, 126562), (2023, 0.0, MEDITERRANEAN, NB, 73.72%, 0.27%, 5.15%, 20.86%, 1339400), (2023, 0.0, MEDITERRANEAN, WB, 78.14%, 0.52%, 5.53%, 15.80%, 45814), (2023, 0.0, SCANDINAVIA, NB, 77.04%, 0.39%, 6.03%, 16.54%, 375142), (2023, 0.0, WEST EUROPE, NB, 76.26%, 0.38%, 5.74%, 17.61%, 2322135), (2023, 0.0, WEST EUROPE, WB, 75.83%, 0.39%, 5.46%, 18.32%, 513), (2023, 7.0, MEDITERRANEAN, WB, 73.41%, 0.61%, 8.72%, 17.26%, 2144), (2023, 7.0, WEST EUROPE, WB, 75.41%, 0.13%, 6.08%, 18.38%, 789), (2023, 8.0, MEDITERRANEAN, WB, 65.68%, 1.03%, 11.13%, 22.17%, 28376), (2023, 8.0, WEST EUROPE, WB, 76.19%, 0.56%, 5.06%, 18.18%, 1067), (2023, 10.0, MEDITERRANEAN, WB, 70.08%, 1.24%, 8.44%, 20.25%, 889), (2023, 10.0, WEST EUROPE, WB, 75.37%, 0.37%, 4.78%, 19.49%, 272), (2023, 14.0, MEDITERRANEAN, WB, 78.03%, 0.32%, 4.34%, 17.31%, 2167), (2023, 14.0, WEST EUROPE, WB, 74.48%, 0.00%, 4.55%, 20.98%, 286)]

Closer Look into the N American Market as well

In [78]:
pivot_df_merged_N_Am = pivot_df_merged[pivot_df_merged['CORP_GEOG_CONTINENT_NM'] == 'NORTH AMERICA']

df_pivot_groupby_First_N_Am_Coun_Acft_typ = group_pivot(pivot_df_merged_N_Am, ['GMT_UPLIFT_YR','FIRST_SEATS_QTY','CORP_GEOG_CTRY_GRP_NM','WB_NB_CAT'])
group_pivot_to_lookup(df_pivot_groupby_First_N_Am_Coun_Acft_typ)

Empty DataFrame
Columns: []
Index: [(2019, 0.0, CANADA, WB, 80.72%, 0.30%, 3.90%, 15.08%, 86264), (2019, 0.0, USA, WB, 80.58%, 0.25%, 3.59%, 15.58%, 109032), (2019, 7.0, USA, WB, 67.81%, 4.43%, 8.00%, 19.77%, 3976), (2019, 8.0, CANADA, WB, 75.45%, 1.71%, 4.26%, 18.59%, 38347), (2019, 8.0, USA, WB, 72.68%, 3.23%, 4.78%, 19.31%, 130510), (2019, 12.0, CANADA, WB, 72.92%, 4.48%, 2.80%, 19.80%, 1854), (2019, 12.0, USA, WB, 67.03%, 5.38%, 5.91%, 21.68%, 97999), (2019, 14.0, CANADA, WB, 74.66%, 3.68%, 2.96%, 18.71%, 109905), (2019, 14.0, USA, WB, 68.77%, 4.56%, 5.47%, 21.21%, 1708905), (2019, 17.0, CANADA, WB, 68.57%, 6.83%, 4.04%, 20.56%, 11160), (2019, 17.0, USA, WB, 64.83%, 6.93%, 6.39%, 21.85%, 58994), (2023, 0.0, CANADA, WB, 80.64%, 0.20%, 3.01%, 16.15%, 131420), (2023, 0.0, USA, WB, 78.85%, 0.15%, 4.11%, 16.89%, 548999), (2023, 7.0, CANADA, WB, 76.41%, 0.22%, 3.86%, 19.51%, 28290), (2023, 7.0, NORTH AMERICA OTHER, WB, 66.34%, 3.17%, 10.72%, 19.76%, 19098), (2023, 7.0, USA, WB, 72.64%, 3.33%, 5.41%, 18.62%, 441202), (2023, 8.0, CANADA, WB, 74.97%, 0.36%, 4.78%, 19.90%, 28801), (2023, 8.0, USA, WB, 64.83%, 3.29%, 6.59%, 25.29%, 345654), (2023, 10.0, CANADA, WB, 74.07%, 0.00%, 2.27%, 23.67%, 883), (2023, 10.0, NORTH AMERICA OTHER, WB, 64.95%, 4.15%, 10.39%, 20.51%, 14926), (2023, 10.0, USA, WB, 69.96%, 3.78%, 6.34%, 19.92%, 13363), (2023, 14.0, CANADA, WB, 77.16%, 0.21%, 3.54%, 19.09%, 1865), (2023, 14.0, USA, WB, 72.57%, 3.43%, 5.17%, 18.83%, 567918)]

### OUR PROPOSED LOOKUP TABLE

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of customers as columns.

In [81]:
# Create final production lookup tables. One each for short haul and long haul, defined by selling classes.

In [81]:
# create bins for first class seats.
bins = [-3,-1,0,6,10,17]
labels = ['SH','0','1-6','7-10','11-17']
# There are no flights with 1-6 first class seats. SH is included only for later use - no aircraft has negative first class seat quantities.

pd.options.mode.chained_assignment = None  # Disable the warning

pivot_df_all_bins = pivot_df_merged.copy()


pivot_df_all_bins['FIRST_SEATS_BINS'] = pd.cut(pivot_df_all_bins['FIRST_SEATS_QTY'],
                                               bins=bins,labels=labels)

# if short haul, then any first class seats are not 'real' First seats, so we can change it back to 'SH' for Short haul.

pivot_df_all_bins = pivot_df_all_bins.copy()

pivot_df_all_bins.loc[(pivot_df_all_bins['HAUL']=='S'),'FIRST_SEATS_BINS']='SH'

### Short haul

In [82]:
pivot_df_short_haul = pivot_df_all_bins.loc[pivot_df_all_bins['HAUL']=='S']

In [83]:
df_short_haul_pivot = group_pivot(pivot_df_short_haul,['FIRST_SEATS_BINS','CORP_GEOG_CTRY_GRP_NM'])

In [84]:
df_short_haul_pivot.dropna(inplace=True)

In [85]:
df_short_haul_pivot

,FIRST_SEATS_BINS,CORP_GEOG_CTRY_GRP_NM,Not_Eligible,Tier_1,Tier_2,Tier_3,TOTAL_PAX
0,SH,BENELUX,74.83%,0.64%,7.46%,17.07%,662824.0
1,SH,EASTERN EUROPE,80.57%,0.26%,4.35%,14.82%,993039.0
2,SH,IRELAND RO,75.65%,0.61%,6.53%,17.22%,328767.0
3,SH,MEDITERRANEAN,74.59%,0.33%,5.33%,19.75%,2437740.0
4,SH,MIDDLE EAST OTHER,78.34%,0.14%,3.62%,17.90%,22495.0
5,SH,NORTH AFRICA,76.10%,0.28%,3.77%,19.85%,98391.0
6,SH,SCANDINAVIA,76.83%,0.49%,6.43%,16.25%,905469.0
7,SH,UK,76.81%,0.39%,5.34%,17.47%,2314153.0
8,SH,WEST EUROPE,75.82%,0.45%,6.30%,17.43%,4935510.0


### Long haul

In [86]:
pivot_df_long_haul = (pivot_df_all_bins.loc[pivot_df_all_bins['HAUL']=='L']).copy()

In [87]:
# Let's group Bermuda and USA into one country group for the purposes of the breakdown.
def group_usa_bermuda(category):
    if category in ('United States', 'Bermuda'):
        return 'USA & Bermuda'
    else:
        return category
    
# df.loc[(df['BA_PAX_TIER'].isin(ba_emerald_list))|(df['ONEWORLD_TIER']=='EMER'),'STATUS']='Emerald'

pivot_df_long_haul.loc[(pivot_df_long_haul['COUNTRY_NM'].isin(['United States','Bermuda'])),'COUNTRY_NM',]='USA & Bermuda'

In [88]:
pivot_df_long_haul.head(1)

,Skew_Id,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TIME,ACT_DEP_TML_CD,DISCHARGE_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,HAUL,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passengers,Not eligible ratio,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio,C,F,J,M,W,FIRST_SEATS_BINS
0,2019-03-31103,103,2019-03-31,17:30:00,5,YYC,788,B8,CA,Canada,CANADA,NORTH AMERICA,WB,0.0,35.0,25.0,154.0,2019,L,154,1,9,33,197,0.781726,0.005076,0.045685,0.167513,0,0,29,143,25,0


In [89]:
north_america_pivot = pivot_df_long_haul[pivot_df_long_haul['CORP_GEOG_CONTINENT_NM']=='NORTH AMERICA']
other_continents_pivot = pivot_df_long_haul[pivot_df_long_haul['CORP_GEOG_CONTINENT_NM']!='NORTH AMERICA']


In [90]:
# We want to break down by FIRST SEATS QTY and Continent. In North America, we want to also break down by country.

pivot_north_america = group_pivot(north_america_pivot,['FIRST_SEATS_BINS','CORP_GEOG_CONTINENT_NM','COUNTRY_NM'])
pivot_long_haul_others = group_pivot(other_continents_pivot,['FIRST_SEATS_BINS','CORP_GEOG_CONTINENT_NM'])

df_long_haul_pivot = pd.concat([pivot_north_america,pivot_long_haul_others]).sort_values(by=['FIRST_SEATS_BINS','CORP_GEOG_CONTINENT_NM'])
df_long_haul_pivot['COUNTRY_NM'].fillna('All',inplace=True)
df_long_haul_pivot = df_long_haul_pivot[df_long_haul_pivot['FIRST_SEATS_BINS'].isin(['0','7-10','11-17'])]
df_long_haul_pivot

,FIRST_SEATS_BINS,CORP_GEOG_CONTINENT_NM,COUNTRY_NM,Not_Eligible,Tier_1,Tier_2,Tier_3,TOTAL_PAX
7,0,AFRICA,All,79.62%,0.37%,4.27%,15.74%,190877.0
8,0,ASIA FAR EAST,All,80.97%,0.24%,3.31%,15.47%,127412.0
9,0,ASIA SOUTH EAST AND CENTRAL,All,80.11%,0.21%,3.11%,16.57%,362863.0
10,0,AUSTRALASIA PACIFIC,All,79.73%,0.29%,4.16%,15.82%,49955.0
11,0,EUROPE EXC UK,All,78.25%,0.70%,6.22%,14.82%,125142.0
12,0,MIDDLE EAST,All,78.46%,0.44%,5.40%,15.70%,73369.0
2,0,NORTH AMERICA,Canada,80.67%,0.24%,3.36%,15.73%,217684.0
3,0,NORTH AMERICA,USA & Bermuda,79.13%,0.16%,4.03%,16.68%,658031.0
13,0,SOUTH AMERICA INC CARIBBEAN,All,80.81%,0.19%,3.16%,15.84%,163611.0
21,7-10,AFRICA,All,71.01%,3.14%,5.08%,20.77%,79330.0


In [91]:
df_long_haul_pivot

,FIRST_SEATS_BINS,CORP_GEOG_CONTINENT_NM,COUNTRY_NM,Not_Eligible,Tier_1,Tier_2,Tier_3,TOTAL_PAX
7,0,AFRICA,All,79.62%,0.37%,4.27%,15.74%,190877.0
8,0,ASIA FAR EAST,All,80.97%,0.24%,3.31%,15.47%,127412.0
9,0,ASIA SOUTH EAST AND CENTRAL,All,80.11%,0.21%,3.11%,16.57%,362863.0
10,0,AUSTRALASIA PACIFIC,All,79.73%,0.29%,4.16%,15.82%,49955.0
11,0,EUROPE EXC UK,All,78.25%,0.70%,6.22%,14.82%,125142.0
12,0,MIDDLE EAST,All,78.46%,0.44%,5.40%,15.70%,73369.0
2,0,NORTH AMERICA,Canada,80.67%,0.24%,3.36%,15.73%,217684.0
3,0,NORTH AMERICA,USA & Bermuda,79.13%,0.16%,4.03%,16.68%,658031.0
13,0,SOUTH AMERICA INC CARIBBEAN,All,80.81%,0.19%,3.16%,15.84%,163611.0
21,7-10,AFRICA,All,71.01%,3.14%,5.08%,20.77%,79330.0


In [92]:
df_short_haul_pivot

,FIRST_SEATS_BINS,CORP_GEOG_CTRY_GRP_NM,Not_Eligible,Tier_1,Tier_2,Tier_3,TOTAL_PAX
0,SH,BENELUX,74.83%,0.64%,7.46%,17.07%,662824.0
1,SH,EASTERN EUROPE,80.57%,0.26%,4.35%,14.82%,993039.0
2,SH,IRELAND RO,75.65%,0.61%,6.53%,17.22%,328767.0
3,SH,MEDITERRANEAN,74.59%,0.33%,5.33%,19.75%,2437740.0
4,SH,MIDDLE EAST OTHER,78.34%,0.14%,3.62%,17.90%,22495.0
5,SH,NORTH AFRICA,76.10%,0.28%,3.77%,19.85%,98391.0
6,SH,SCANDINAVIA,76.83%,0.49%,6.43%,16.25%,905469.0
7,SH,UK,76.81%,0.39%,5.34%,17.47%,2314153.0
8,SH,WEST EUROPE,75.82%,0.45%,6.30%,17.43%,4935510.0


In [93]:
df_short_haul_pivot = df_short_haul_pivot.rename(columns={'CORP_GEOG_CTRY_GRP_NM':'Country Group (SH)/ Continent Group (LH)'})
df_short_haul_pivot['COUNTRY_NM'] = 'All'

df_long_haul_pivot = df_long_haul_pivot.rename(columns={'CORP_GEOG_CONTINENT_NM':'Country Group (SH)/ Continent Group (LH)'})



# final_output_lookup_table 

In [94]:
df_short_haul_pivot

,FIRST_SEATS_BINS,Country Group (SH)/ Continent Group (LH),Not_Eligible,Tier_1,Tier_2,Tier_3,TOTAL_PAX,COUNTRY_NM
0,SH,BENELUX,74.83%,0.64%,7.46%,17.07%,662824.0,All
1,SH,EASTERN EUROPE,80.57%,0.26%,4.35%,14.82%,993039.0,All
2,SH,IRELAND RO,75.65%,0.61%,6.53%,17.22%,328767.0,All
3,SH,MEDITERRANEAN,74.59%,0.33%,5.33%,19.75%,2437740.0,All
4,SH,MIDDLE EAST OTHER,78.34%,0.14%,3.62%,17.90%,22495.0,All
5,SH,NORTH AFRICA,76.10%,0.28%,3.77%,19.85%,98391.0,All
6,SH,SCANDINAVIA,76.83%,0.49%,6.43%,16.25%,905469.0,All
7,SH,UK,76.81%,0.39%,5.34%,17.47%,2314153.0,All
8,SH,WEST EUROPE,75.82%,0.45%,6.30%,17.43%,4935510.0,All


In [95]:
final_lookup_table = pd.concat([df_long_haul_pivot,df_short_haul_pivot]).reset_index(drop=True)

### FINAL LOOKUP TABLE

The final table first separates flights by haul, calculated using booking classes. Any flight with F, J, or W classes available is long-haul by BA RevMan definition. Any flight with C class is short-haul by RevMan definition.

For short-haul flights, it then lookups against the country group. For long-haul flights, it first lookups against the number of first class seats followed by the continent. There were significant differences identified between the Canada and the US & Bermuda, so the North American continent group is split again into country. This can be done as the number of flights to both the US and Canada is very high relative to any other single other country. There is not enough data to split by country for other continents.

Additionally, this dataset is 'complete' - that is, for any long-haul destination, the continent is shown for all values of first class seats, so any new destination or new aircraft deployment (e.g. addition of First to a new route) is catered for in the table.

There is no split by Club Europe seats for short-haul. The curtain moves dynamically throughout the booking process, so the number of Club Europe seats on each flight will not be known to the lounge team when they make their recommendations on lounge sizing.

In [97]:
final_lookup_table

,FIRST_SEATS_BINS,Country Group (SH)/ Continent Group (LH),COUNTRY_NM,Not_Eligible,Tier_1,Tier_2,Tier_3,TOTAL_PAX
0,0,AFRICA,All,79.62%,0.37%,4.27%,15.74%,190877.0
1,0,ASIA FAR EAST,All,80.97%,0.24%,3.31%,15.47%,127412.0
2,0,ASIA SOUTH EAST AND CENTRAL,All,80.11%,0.21%,3.11%,16.57%,362863.0
3,0,AUSTRALASIA PACIFIC,All,79.73%,0.29%,4.16%,15.82%,49955.0
4,0,EUROPE EXC UK,All,78.25%,0.70%,6.22%,14.82%,125142.0
5,0,MIDDLE EAST,All,78.46%,0.44%,5.40%,15.70%,73369.0
6,0,NORTH AMERICA,Canada,80.67%,0.24%,3.36%,15.73%,217684.0
7,0,NORTH AMERICA,USA & Bermuda,79.13%,0.16%,4.03%,16.68%,658031.0
8,0,SOUTH AMERICA INC CARIBBEAN,All,80.81%,0.19%,3.16%,15.84%,163611.0
9,7-10,AFRICA,All,71.01%,3.14%,5.08%,20.77%,79330.0


### Addtional analysis Jamie